# 04_01: Creating a GraphRAG pipeline with LangChain to query your data

In [ ]:
URI = "..."
USER = "neo4j"
PWD = "..."
OPENAI_API_KEY = "..."

In [ ]:
import os

from langchain_core.documents import Document
from langchain_community.graphs.graph_document import GraphDocument
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain.prompts import PromptTemplate

In [ ]:
graph = Neo4jGraph(url=URI, username=USER, password=PWD)
graph.query('MATCH (n) RETURN COUNT(n) as TOTAL')

In [ ]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to
answer questions about Portugal.
Convert the user's question based on the schema.
When you are presented with query properties such as id's like "politics of portugal",
be sure to convert the first letter to capital case, such as "Politics of Portugal"
before you run the Cypher query.

If you are asked a generic question asking you to describe an node based on its
id, find all nodes that match that id and then return a description based on all
other nodes that are connected to it.  DO NOT ASSUME A PARTICULAR NODE TYPE.

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

In [ ]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model_name="gpt-4o")
llm_transformer = LLMGraphTransformer(llm=llm)

cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True
)

In [ ]:
cypher_chain.invoke({"query": "What regions does the Macaronesian Archipelagos include?"})

In [ ]:
cypher_chain.invoke({"query": "Tell me about the Macaronesian Archipelagos."})

In [ ]:
cypher_chain.invoke({"query": "Tell me about transport in Portugal."})